In [223]:
# import sys
# !{sys.executable} -m pip install lightfm
import random
import itertools

import pickle

import numpy as np
import pandas as pd
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
from scipy import sparse
import random
import lightfm
from lightfm import LightFM, cross_validation
from lightfm.evaluation import precision_at_k, auc_score
from sklearn.metrics.pairwise import cosine_similarity

from surprise import Dataset
from surprise import Reader
from surprise import SVDpp
from collections import defaultdict

In [224]:
v = 50

In [225]:
# df_playlist = pd.read_csv('./data/spotify_dataset.csv', error_bad_lines=False, warn_bad_lines=False, skiprows=lambda i: i>0 and random.random() > p)
df_playlist = pd.read_csv('./data/spotify_dataset.csv', error_bad_lines=False, warn_bad_lines=False, skiprows=lambda i: i>0 and (i % v != 0))

In [226]:
df_playlist.shape

(257837, 4)

In [227]:
df_playlist.head()

,user_id,"""artistname""","""trackname""","""playlistname"""
0,9cc0cfd4d7d7885102480dd99e7a90d6,Cocktail Slippers,Sentenced To Love,HARD ROCK 2010
1,9cc0cfd4d7d7885102480dd99e7a90d6,Bruce Springsteen,The River,IOW 2012
2,07f0fc3be95dcd878966b1f9572ff670,C418,0x10c - 02,Chill out
3,07f0fc3be95dcd878966b1f9572ff670,Emancipator,Greenland,Chill out
4,07f0fc3be95dcd878966b1f9572ff670,Trifonic,Santa Rosa,Chill out


In [228]:
df_playlist.columns = df_playlist.columns.str.replace('"', '')
df_playlist.columns = df_playlist.columns.str.replace('name', '')
df_playlist.columns = df_playlist.columns.str.replace(' ', '')
df_playlist.columns
df_playlist.head()

,user_id,artist,track,playlist
0,9cc0cfd4d7d7885102480dd99e7a90d6,Cocktail Slippers,Sentenced To Love,HARD ROCK 2010
1,9cc0cfd4d7d7885102480dd99e7a90d6,Bruce Springsteen,The River,IOW 2012
2,07f0fc3be95dcd878966b1f9572ff670,C418,0x10c - 02,Chill out
3,07f0fc3be95dcd878966b1f9572ff670,Emancipator,Greenland,Chill out
4,07f0fc3be95dcd878966b1f9572ff670,Trifonic,Santa Rosa,Chill out


In [229]:
# Adding test user and seeing what the output will be.
temp_user_id = '00000000000000000000000000000000'
temp_user_artists = [
    'Eminem', 'Kanye West', '50 Cent', 'Drake', 'Kendrick Lamar', 'Future', 
    'Lil Baby', 'Snoop Dogg', 'Post Malone', 'Lil Wayne', 'Kid Cudi']
temp_track_name = 'testtrackname'
temp_playlist_name = 'testplaylist'

new_rows = []
for i in range(5000):
    temp_artist = random.choice(temp_user_artists)
    new_row = {
        'user_id': temp_user_id,
        'artist': temp_artist,
        'track': temp_track_name,
        'playlist': temp_playlist_name
    }
    new_rows.append(new_row)

temp_df = pd.DataFrame(new_rows) 
df_playlist = pd.concat([df_playlist, temp_df], ignore_index = True)
df_last = df_playlist.tail(10)
df_last

df_playlist.loc[df_playlist['user_id'] == temp_user_id]

,user_id,artist,track,playlist
257837,00000000000000000000000000000000,Snoop Dogg,testtrackname,testplaylist
257838,00000000000000000000000000000000,Lil Baby,testtrackname,testplaylist
257839,00000000000000000000000000000000,Kid Cudi,testtrackname,testplaylist
257840,00000000000000000000000000000000,Future,testtrackname,testplaylist
257841,00000000000000000000000000000000,Post Malone,testtrackname,testplaylist
...,...,...,...,...
262832,00000000000000000000000000000000,50 Cent,testtrackname,testplaylist
262833,00000000000000000000000000000000,50 Cent,testtrackname,testplaylist
262834,00000000000000000000000000000000,Post Malone,testtrackname,testplaylist
262835,00000000000000000000000000000000,Kanye West,testtrackname,testplaylist


In [230]:
# Keeping artists that appear with a minimum frequency. 
# Only keeping artists with high frequency than 50

In [231]:
df_playlist = df_playlist.groupby('artist').filter(lambda x : len(x) >= 50)
# df_playlist
df_playlist.loc[df_playlist['user_id'] == temp_user_id]

,user_id,artist,track,playlist
257837,00000000000000000000000000000000,Snoop Dogg,testtrackname,testplaylist
257838,00000000000000000000000000000000,Lil Baby,testtrackname,testplaylist
257839,00000000000000000000000000000000,Kid Cudi,testtrackname,testplaylist
257840,00000000000000000000000000000000,Future,testtrackname,testplaylist
257841,00000000000000000000000000000000,Post Malone,testtrackname,testplaylist
...,...,...,...,...
262832,00000000000000000000000000000000,50 Cent,testtrackname,testplaylist
262833,00000000000000000000000000000000,50 Cent,testtrackname,testplaylist
262834,00000000000000000000000000000000,Post Malone,testtrackname,testplaylist
262835,00000000000000000000000000000000,Kanye West,testtrackname,testplaylist


In [232]:
# Keeping users with at least 10 unique artists in their playlists to lessen
# the impact of COLD START PROBLEM

In [233]:
df_playlist = df_playlist[df_playlist.groupby('user_id').artist.transform('nunique') >= 10]
df_playlist.loc[df_playlist['user_id'] == temp_user_id]

,user_id,artist,track,playlist
257837,00000000000000000000000000000000,Snoop Dogg,testtrackname,testplaylist
257838,00000000000000000000000000000000,Lil Baby,testtrackname,testplaylist
257839,00000000000000000000000000000000,Kid Cudi,testtrackname,testplaylist
257840,00000000000000000000000000000000,Future,testtrackname,testplaylist
257841,00000000000000000000000000000000,Post Malone,testtrackname,testplaylist
...,...,...,...,...
262832,00000000000000000000000000000000,50 Cent,testtrackname,testplaylist
262833,00000000000000000000000000000000,50 Cent,testtrackname,testplaylist
262834,00000000000000000000000000000000,Post Malone,testtrackname,testplaylist
262835,00000000000000000000000000000000,Kanye West,testtrackname,testplaylist


In [234]:
# Grouping by the frequency count for each users artists (# of times that an
# artist appeared in playlists created by a user)

In [235]:
size = lambda x : len(x)
df_freq = df_playlist.groupby(['user_id', 'artist']).agg('size').reset_index().rename(columns={0:'freq'})[['user_id', 'artist', 'freq']].sort_values(['freq'], ascending=False)
df_freq.head()

df_freq.loc[df_freq['user_id'] == temp_user_id]

,user_id,artist,freq
8,00000000000000000000000000000000,Lil Wayne,481
1,00000000000000000000000000000000,Drake,475
10,00000000000000000000000000000000,Snoop Dogg,471
2,00000000000000000000000000000000,Eminem,469
3,00000000000000000000000000000000,Future,460
5,00000000000000000000000000000000,Kendrick Lamar,459
7,00000000000000000000000000000000,Lil Baby,458
4,00000000000000000000000000000000,Kanye West,445
9,00000000000000000000000000000000,Post Malone,434
0,00000000000000000000000000000000,50 Cent,433


In [236]:
# Creating a data frame for artists and artist id
# Artist to artist id mapping

In [237]:
df_artist = pd.DataFrame(df_freq['artist'].unique())
df_artist = df_artist.reset_index()
df_artist = df_artist.rename(columns={'index':'artist_id', 0:'artist'})
df_artist.head()

,artist_id,artist
0,0,Lil Wayne
1,1,Drake
2,2,Snoop Dogg
3,3,Eminem
4,4,Future


In [238]:
df_artist.shape

(840, 2)

In [239]:
# Adding artist_id to the frequence data frame
df_freq = pd.merge(df_freq, df_artist, how='inner', on='artist')

In [240]:
#########
# Using the lightfm library to run a traditional MF (matrix factorization) model
# since the dataset doesn't include any user or artist features
# the library allows builing of a hybrid model as well. (CF AND CBF?)
# examples: https://github.com/lyst/lightfm/blob/master/examples/

In [241]:
## Parameter Tuning
def sample_hyperparameters():
    """
    Yield possible hyperparameter choices.
    """

    while True:
        yield {
            "no_components": np.random.randint(16, 64),
            "learning_schedule": np.random.choice(["adagrad", "adadelta"]),
            "loss": np.random.choice(["bpr", "warp", "warp-kos"]),
            "learning_rate": np.random.exponential(0.05),
            "item_alpha": np.random.exponential(1e-8),
            "user_alpha": np.random.exponential(1e-8),
            "max_sampled": np.random.randint(5, 15),
            "num_epochs": np.random.randint(5, 50),
        }

In [242]:
# Don't know what the
# Creating an interaction matrix df from transactional type interactions
def create_interaction_matrix(df, user_col, item_col, rating_col, norm=False, threshold=None):
    interactions = df.groupby([user_col, item_col])[rating_col].sum().unstack().reset_index().fillna(0).set_index(user_col)
    if norm:
        interactions = interactions.applymap(lambda x: 1 if x > threshold else 0)
    return interactions

In [243]:
# Function to create a user dictionary based on their index and number in
# interaction dataset
def create_user_dict(interactions):
    user_id = list(interactions.index)
    user_dict = {}
    counter = 0 
    for i in user_id:
        user_dict[i] = counter
        counter += 1
    return user_dict

In [244]:
# Function to create an item dictionary based on their item_id and item name
def create_item_dict(df, id_col, name_col):
    item_dict = {}
    for i in range(df.shape[0]):
        item_dict[(df.loc[i,id_col])] = df.loc[i, name_col]
    return item_dict

In [245]:
# Runs the matrix factorization algorithm and returns the trained model
def runMF(interactions, n_components=30, loss='warp', k=15, epoch=30, n_jobs = 4):
#     x = sparse.csr_matrix(interactions.values)
    model = LightFM(no_components = n_components, loss=loss, k=k)
    model.fit(interactions, epochs=epoch, num_threads = n_jobs)
    return model

In [246]:
# Parameter Tuning
def random_search(train, test, num_samples=50, num_threads=4):
    for hyperparams in itertools.islice(sample_hyperparameters(), num_samples):
        num_epochs = hyperparams.pop("num_epochs")
        
        model = LightFM(**hyperparams)
        model.fit(train, epochs=num_epochs, num_threads=num_threads)
        
        test_auc = auc_score(model, test, train_interactions=train, num_threads=num_threads).mean()
        train_auc = auc_score(model, train, num_threads=num_threads).mean()
        
        hyperparams["num_epochs"] = num_epochs
        
        yield (test_auc, train_auc, hyperparams, model)

In [247]:
# Produces user recommendations, prints the list of items the user already 
# prefers, prints the list of N recommended items which user will hopefully 
# prefer.
def sample_recommendation_user(model, interactions, user_id, user_dict, item_dict, threshold = 0, nrec_items = 10, show = True):
    n_users, n_items = interactions.shape # indices
    user_x = user_dict[user_id] # getting the specific user
    scores = pd.Series(model.predict(user_x, np.arange(n_items))) # model prediction
    scores.index = interactions.columns
    scores = list(pd.Series(scores.sort_values(ascending=False).index))
    
    known_items = list(pd.Series(interactions.loc[user_id,:]
                                 [interactions.loc[user_id,:] > threshold].index).sort_values(ascending=False))
    
    scores = [x for x in scores if x not in known_items]
    return_score_list = scores[0:nrec_items]
    known_items = list(pd.Series(known_items).apply(lambda x: item_dict[x]))
    scores = list(pd.Series(return_score_list).apply(lambda x: item_dict[x]))
    if show == True:
        print("Known Likes:")
        counter = 1
        for i in known_items:
            print(str(counter) + '- ' + i)
            counter+=1

        print("\n Recommended Items:")
        counter = 1
        for i in scores:
            print(str(counter) + '- ' + i)
            counter+=1
    return return_score_list

In [248]:
# Prepping the model inputs

In [249]:
# Create interaction matrix
interactions = create_interaction_matrix(df=df_freq, user_col="user_id", item_col="artist_id", rating_col='freq', norm=False, threshold=None)
interactions.head()

artist_id,0,1,2,3,4,5,6,7,8,9,...,830,831,832,833,834,835,836,837,838,839
user_id,,,,,,,,,,,,,,,,,,,,,
00000000000000000000000000000000,481.0,475.0,471.0,469.0,460.0,459.0,458.0,445.0,434.0,433.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
00123e0f544dee3ab006aa7f1e5725a7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
00287ecf491882a40ff34b0fd75a5b16,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
007cab633671b7475e317587ebb31d8c,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
00909b16e60501d9da9acf7bd2c5b00d,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [250]:
## NEW
# Prep the data for surprise. 
# surprise_df = df_freq.copy(deep=True)
# for index, row in surprise_df.iterrows():
#     if row['freq'] > 200:
#         surprise_df.at[index, 'freq'] = 200

In [251]:
# surprise_df.head()

In [252]:
# def get_top_n(user_id, predictions, n=10):
#     """Return the top-N recommendation for each user from a set of predictions.

#     Args:
#         predictions(list of Prediction objects): The list of predictions, as
#             returned by the test method of an algorithm.
#         n(int): The number of recommendation to output for each user. Default
#             is 10.

#     Returns:
#     A dict where keys are user (raw) ids and values are lists of tuples:
#         [(raw item id, rating estimation), ...] of size n.
#     """

#     # First map the predictions to each user.
#     top_n = defaultdict(list)
#     for uid, iid, true_r, est, _ in predictions:
#         top_n[user_id].append((iid, est))

#     # Then sort the predictions for each user and retrieve the k highest ones.
#     for uid, user_ratings in top_n.items():
#         user_ratings.sort(key=lambda x: x[1], reverse=True)
#         top_n[uid] = user_ratings[:n]

#     return top_n

In [253]:
# reader = Reader(rating_scale=(0,200))
# data = Dataset.load_from_df(surprise_df[["user_id", "artist_id", "freq"]], reader)
# trainingSet = data.build_full_trainset()

# algo = SVD()
# algo.fit(trainingSet)

# # Than predict ratings for all pairs (u, i) that are NOT in the training set.
# testset = trainingSet.build_anti_testset()
# predictions = algo.test(testset)

In [254]:
# top_n = get_top_n(predictions, n=10)

# # Print the recommended items for each user

# for uid, user_ratings in top_n.items():
#     print(uid, [iid for (iid, _) in user_ratings])

In [255]:
# artists_dict = create_item_dict(df=df_artist, id_col="artist_id", name_col="artist")
# artists_dict

## NEW END

In [256]:
interactions.shape

(2469, 840)

In [257]:
# temp

In [258]:
# Create user dict
user_dict = create_user_dict(interactions=interactions)

In [259]:
# Create item dict
artists_dict = create_item_dict(df=df_artist, id_col="artist_id", name_col="artist")

In [260]:
# Train-test split
x = sparse.csr_matrix(interactions.values)
train, test = lightfm.cross_validation.random_train_test_split(x, test_percentage=0.2, random_state=None)

In [261]:
# How does a matrix factorization model work?
# https://developers.google.com/machine-learning/recommendation/collaborative/matrix

In [262]:
# Train the matrix factorization model
# Can do hyper-parameter tuning for better results
# %time
model = runMF(interactions = train, n_components=30, loss='warp', k=15, epoch=30, n_jobs=4)

# file = open('cf_model.pickle', 'rb')
# model = pickle.load(file)
# file.close()

In [263]:
# ## New Code
# (test_auc, train_auc, hyperparams, model) = max(random_search(train, test, num_threads=4), key=lambda x: x[0])

# print("Best Test AUC Score: {}\nTrain AUC Score: {}\nParams: {}".format(test_auc, train_auc, hyperparams))

# ## End New Code

In [264]:
# Evaluation metrics
# Compute AUC score for train set
# Original
# train_auc = auc_score(model, train, num_threads=4).mean()
# print('Train AUC: %s' % train_auc)

In [265]:
# Compute AUC score test set
# the parameter train_interactions allows you to exlude known positives in
# training set from the predictions and score calculations
# this is to avoid re-recommending the items the user has already interacted with
# Original
# test_auc = auc_score(model, test, train_interactions=train, num_threads=4).mean()
# print('Test AUC: %s' % test_auc)

In [266]:
train_precision = precision_at_k(model, train, k=10).mean()
test_precision = precision_at_k(model, test, k=10, train_interactions=train).mean()

In [267]:
print('train Precision %.2f, test Precision %.2f.' % (train_precision, test_precision))

train Precision 0.47, test Precision 0.03.


In [268]:
random_cell = interactions.sample()
random_user_id = random_cell.index[0]

rec_list = sample_recommendation_user(model = model, 
                                      interactions = interactions, 
                                      user_id = temp_user_id, 
                                      user_dict = user_dict,
                                      item_dict = artists_dict, 
                                      threshold = 0,
                                      nrec_items = 10,
                                      show = True)

Known Likes:
1- Kid Cudi
2- 50 Cent
3- Post Malone
4- Kanye West
5- Lil Baby
6- Kendrick Lamar
7- Future
8- Eminem
9- Snoop Dogg
10- Drake
11- Lil Wayne

 Recommended Items:
1- JAY Z
2- Nas
3- Alicia Keys
4- Dr. Dre
5- Pharrell Williams
6- Wiz Khalifa
7- 2Pac
8- Busta Rhymes
9- Beyoncé
10- The Weeknd


In [269]:
# print(type(test_auc))

In [270]:
## Trying to export model


with open('cf_model.pickle', 'wb') as file:
    pickle.dump(model, file)

## END export

In [271]:
## Load model get prediction

file = open('cf_model.pickle', 'rb')
pickled_model = pickle.load(file)
file.close()

rec_list = sample_recommendation_user(model = pickled_model, 
                                      interactions = interactions, 
                                      user_id = temp_user_id, 
                                      user_dict = user_dict,
                                      item_dict = artists_dict, 
                                      threshold = 0,
                                      nrec_items = 10,
                                      show = True)

Known Likes:
1- Kid Cudi
2- 50 Cent
3- Post Malone
4- Kanye West
5- Lil Baby
6- Kendrick Lamar
7- Future
8- Eminem
9- Snoop Dogg
10- Drake
11- Lil Wayne

 Recommended Items:
1- JAY Z
2- Nas
3- Alicia Keys
4- Dr. Dre
5- Pharrell Williams
6- Wiz Khalifa
7- 2Pac
8- Busta Rhymes
9- Beyoncé
10- The Weeknd
